## <font color=yellow>1. model version alias를 활용한 model load</font>

In [11]:
import mlflow
mlflow.set_tracking_uri('http://127.0.0.1:5000')

In [12]:
model_name = "hellomlflow"
model_version_alias = "staging"

model_uri = f"models:/{model_name}@{model_version_alias}"
loaded_model = mlflow.pyfunc.load_model(model_uri)

In [13]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: mlflow-artifacts:/412400933482959348/models/m-b42059930aa048bb92ef4ee5ebe7b139/artifacts
  flavor: mlflow.sklearn
  run_id: 4cb11270667d48e3bb03f8ee8b97c24e

In [14]:
import pandas as pd
from sklearn.metrics import accuracy_score

test_df = pd.read_csv('dataset_preprocessing.csv', index_col=0)
label = test_df['Survived']
test_df = test_df.drop(['Survived'], axis=1)
test_df.head()

,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_N,Embarked_Q,Embarked_S
0,0.271174,0.125,0.0,0.014151,False,False,True,False,True,False,False,False,True
1,0.472229,0.125,0.0,0.139136,True,False,False,True,False,True,False,False,False
2,0.321438,0.000,0.0,0.015469,False,False,True,True,False,False,False,False,True
3,0.434531,0.125,0.0,0.103644,True,False,False,True,False,False,False,False,True
4,0.434531,0.000,0.0,0.015713,False,False,True,False,True,False,False,False,True


In [15]:
y_prd = loaded_model.predict(pd.DataFrame(test_df))
accuracy_score(y_prd, label)

0.9382716049382716

## <font color=yellow>2. mlflow를 이용한 API Serving</font>

In [16]:
import pandas as pd
import requests

host = "127.0.0.1"
url = f"http://{host}:5002/invocations"

json_data = {
    "dataframe_split": test_df[:10].to_dict(orient="split")
    }

response = requests.post(url, json=json_data)
print(f"\nPyfunc 'predict_interval':\n${response.json()}")


Pyfunc 'predict_interval':
${'predictions': [0, 1, 1, 1, 0, 0, 0, 0, 1, 1]}


In [17]:
json_data = {
    "dataframe_split": test_df[20:40].to_dict(orient="split")
    }

response = requests.post(url, json=json_data)
print(f"\nPyfunc 'predict_interval':\n${response.json()}")


Pyfunc 'predict_interval':
${'predictions': [0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]}
